Ant Colony - Laboratório IA
Pedro Santos e Vinicius Nascimento

In [103]:
from City import City
from Ridge import Ridge
import numpy as np
import random
import math
import csv

# Variáveis Globais
alpha = 1
beta = 1
Q_rate = 1.0
p_rate = 0,5
cities_array = []
ants = []
ridges = np.empty((6, 6), dtype=Ridge) # Vai da cidade 0 até a cidade 32

In [104]:
# Construindo o vetor de cidades, e instanciando os objetos City para salvar as coordenadas de cada cidade

def build_cities():
    global cities_array
    with open('C:\\Users\\pedro\\OneDrive\\Documentos\\CEFETMG\\2023.1\\Inteligência Artificial\\Laboratório\\AI-Algorithms\\Ant Colony - Teste\\Colonia.csv', "r") as csvfile:
        filereader = csv.reader(csvfile, delimiter=';')
        next(filereader, None)
        for row in filereader:
            id, x, y = row
            id = int(id)
            x = float(x)
            y = float(y)
            cities_array.append(City(id, x, y))

In [105]:
# Construindo o vetor de formigas, cada posição do vetor equivale a uma formiga, e guarda as cidades vistadas por ela

def build_ants_initial_position():
    global ants, cities_array
    for i in cities_array:
        ants.append([i.get_id()])

In [106]:
# Cálculo da distância entre duas cidades usando função euclidiana

def distance(x1:float,x2:float,y1:float,y2:float) -> float:
    return math.sqrt((x1-x2)**2 + (y1-y2)**2)

In [107]:
# Criando uma matriz de arestas para salvar todas as distâncias entre todas as cidades

def build_ridges():
    global ridges, cities_array

    for i in range(len(ridges)):
        for j in range(len(ridges)):
            if i != j:
                ridges[i][j] = Ridge(i, j, 1, distance(cities_array[i].get_x(), cities_array[j].get_x(), cities_array[i].get_y(), cities_array[j].get_y()))
            else:
                ridges[i][j] = Ridge(i, j, 1, 0)

In [108]:
# Calcular a probabilidade de cada formiga ir para cada cidade

def propability(possible_cities:list, ant_cities:list) -> list :

    sum = 0
    cities_propability = []

    print("Caminho Percorrido: ")
    print(ant_cities)

    print("Possíveis próximas cidades: ")
    print(possible_cities)

    # Cidade atual da formiga
    present_city = ant_cities[-1]

    # Calculando o somatório das possíveis rotas
    for i in range(len(possible_cities)):

        ridge = ridges[present_city][possible_cities[i]]
        ph = ((ridge.get_pheromone())**alpha) * ((ridge.get_heuristic())**beta)
        sum = sum + ph

    # Calculando a propabilidade de cada rota
    for i in range(len(possible_cities)):

        prob = 0

        ridge = ridges[present_city][possible_cities[i]]
        p = ridge.get_pheromone()
        h = ridge.get_heuristic()

        prob = ((p)**alpha) * ((h)**beta) / sum

        cities_propability.append(prob)

    return cities_propability



In [109]:
# Calcular a próxima cidade a ser visitada por cada formiga

def select_next_city():
    global ants, cities_array, ridges

    # Printando a solução parcial
    print("Solução Parcial Geral: ")
    print(ants[1:])
    print("\n")

    # Percorrendo o vetor de formigas
    for i in range(1, len(ants), 1):

        possible_cities = []
        next_city = 0        

        # Percorrendo o vetor de cidades para cada fromiga
        for j in range(1, len(cities_array), 1):

             # Checando se a cidade j já foi visitada pela formiga i
            if cities_array[j].get_id() not in ants[i]:
                possible_cities.append(cities_array[j].get_id())
        
        # Calculando a probabilidade para cada cidade não visitada pela formiga i
        probabilities = propability(possible_cities, ants[i])

        # Linkar a cidade com a sua probabilidade de ser visitada, tupla (cidade, probabilidade)
        city_probability = []
        for j in range(0, len(possible_cities), 1):
            city_probability.append((possible_cities[j], probabilities[j]))

        print("Probabilidae de cada rota: ")
        print(city_probability)

        # Aplicando o método da roleta para selecionar a próxima cidade
        sum_city_probability = []
        for j in range(0, len(city_probability), 1):
            if j != 0:
                sum_city_probability.append(city_probability[j][1] + sum_city_probability[j-1])
            else:
                sum_city_probability.append(city_probability[j][1])

        print(sum_city_probability)

        r = random.uniform(0, 1)

        for j in range(0, len(sum_city_probability), 1):
            if r < sum_city_probability[j]:
                next_city = possible_cities[j]
                break

        print("Próxima Cidade Escolhida: ")
        print(next_city, " ", r)
        print("\n")

        # Acrescentando a cidade escolhida na lista de cidades visitadas da formiga i
        ants[i].append(next_city)
        



In [113]:
# Realizar a atualização dos feromônios de cada aresta

def update_pheromones():

    global ants, ridges

    d_cities = 0
    d_route = 0
    distances = []
    distances.append(0)

    # Calcular a distância percorrida por cada formiga
    for i in range(1, len(ants), 1):
        for j in range(len(ants[i])):
            if j != len(ants[i]) - 1:

                # Calculando as distâncias das arestas intermediárias
                origin = ants[i][j]
                destiny = ants[i][j+1]
                d_cities = ridges[origin][destiny].get_heuristic()
                d_route = d_route + d_cities

        # Calculando a distância da cidade final para a cidade inicial
        origin = ants[i][-1]
        destiny = ants[i][0]
        d_cities = ridges[origin][destiny].get_heuristic()
        d_route = d_route + d_cities

            
        distances.append(d_route)

    print("Distâncias Percorridas: ")
    print(distances[1:])
    print("\n")

    # Printando os feromônios antes da atualização
    print("Feromônios Antes: ")
    for i in range(1, 6, 1):
        for j in range(1, 6, 1):
            print(ridges[i][j].get_pheromone(), end=" ")
        print("\n")

    # Realizando a evaporação dos feromônios
    for i in range(1, 6, 1):
        for j in range(1, 6, 1):
            r = ridges[i][j]

            # Calculando taxa de evaporação na aresta ij
            evaporation = (1/2) * r.get_pheromone()

            # Atualizando o valor do feromônio
            r.pheromone = evaporation

    # Calculando a taxa de depósito e realizando o depósito dos feromônios de cada aresta
    for i in range(1, len(ants), 1):
        for j in range(len(ants[i])):
            if j != len(ants[i]) - 1:
                origin = ants[i][j]
                destiny = ants[i][j+1]
                r = ridges[origin][destiny]

                # Calculando a taxa de depósito na aresta j - j+1
                #delta_p = 1 / distances[i]

                # Atualizando o valor do feromônio
                #new_p_value = r.get_pheromone() + delta_p
                #r.pheromone = new_p_value
                

    # Printando os feromônios depois da atualização
    print("Feromônios Depois: ")
    for i in range(1, 6, 1):
        for j in range(1, 6, 1):
            print(ridges[i][j].get_pheromone(), end=" ")
        print("\n")


    

In [114]:
def main():

    global ridges, cities_array, ants

    # Limpando as variáveis
    cities_array.clear(), ants.clear()

    # DEBUG
    # print(cities_array)
    # print(ants)
    # print(ridges)

    # Construindo o vetor de cidades
    build_cities()

    # DEBUG
    # for i in range(len(cities_array)):
    #     print(cities_array[i].get_id(), end=" ")
    #     print(cities_array[i])

    # Construindo o vetor de formigas
    build_ants_initial_position()

    # DEBUG
    # print(*ants)

    # Construindo a matriz de arestas
    build_ridges()

    # DEBUG
    # for i in range(len(ridges)):
    #     for j in range(len(ridges)):
    #         print(ridges[i][j])
    #         print(ridges[i][j].get_city_origin(), ridges[i][j].get_city_destiny(), end="    ")
    #     print("\n")

    # Calcular o caminho percorrido pela formiga, cidade a cidade -> Alterar o range (4 = 5 cidades)
    for i in range(4):
        # Seleciona a próxima cidade a ser visitada para todas as formigas
        select_next_city()

    # DEBUG
    print("Solução Parcial Geral: ")
    print(ants[1:])
    print("\n")

    # Realizar a atualização dos feromônios em cada aresta
    update_pheromones()
    
    # for i in range(1, 6, 1):
    #     for j in range(1, 6, 1):
    #         print(ridges[i][j].get_heuristic(), end=" ")
    #     print("\n")

In [115]:
main()

# 0,2  0,6  0,8  0,9

Solução Parcial Geral: 
[[1], [2], [3], [4], [5]]


Caminho Percorrido: 
[1]
Possíveis próximas cidades: 
[2, 3, 4, 5]
Probabilidae de cada rota: 
[(2, 0.21787473044651565), (3, 0.4357494608930313), (4, 0.22458013169599), (5, 0.12179567696446303)]
[0.21787473044651565, 0.653624191339547, 0.878204323035537, 1.0]
Próxima Cidade Escolhida: 
3   0.4093974223416891


Caminho Percorrido: 
[2]
Possíveis próximas cidades: 
[1, 3, 4, 5]
Probabilidae de cada rota: 
[(1, 0.18062751578809855), (3, 0.18062751578809855), (4, 0.364066397179629), (5, 0.27467857124417394)]
[0.18062751578809855, 0.3612550315761971, 0.7253214287558261, 1.0]
Próxima Cidade Escolhida: 
1   0.1642538380965609


Caminho Percorrido: 
[3]
Possíveis próximas cidades: 
[1, 2, 4, 5]
Probabilidae de cada rota: 
[(1, 0.2346628043052356), (2, 0.1173314021526178), (4, 0.3532142940207653), (5, 0.2947914995213812)]
[0.2346628043052356, 0.3519942064578534, 0.7052085004786187, 0.9999999999999999]
Próxima Cidade Escolhida: 
4   0.36331552

In [47]:
# TESTES
print(len(cities_array))

6
